# Install required libraries for Hugging Face transformers

In [1]:
!pip install transformers accelerate sentencepiece -q

print("Libraries installed successfully!")

Libraries installed successfully!


# Load FLAN-T5-Base model and tokenizer from Hugging Face

In [26]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load FLAN-T5-Base model
model_name = "google/flan-t5-base"

print("Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

print(f"✅ Model loaded successfully on {device}!")

Loading model and tokenizer...
✅ Model loaded successfully on cpu!


# Define a classification function using your master prompt

In [32]:
def classify_message(message):
    """
    Classify customer message with CORRECT output format: Sentiment | Category
    """
    prompt = f"""Task: Analyze the customer message and provide sentiment first, then category.

Categories (choose only one):
- Product Defect: Physical damage, malfunction, missing parts, or quality issues
- Shipping Delay Inquiry: Questions about delivery time, tracking, or late orders
- Feature Request: Suggestions for new features, improvements, or changes
- Positive Feedback: Praise, satisfaction, compliments, or appreciation
- Negative Feedback: Complaints not related to defects or shipping
- Other: General questions, account issues, password resets, or unrelated topics

Examples:
Message: "My phone screen is cracked after just 2 days of use"
Answer: Negative | Product Defect

Message: "When will my order arrive? It's been 2 weeks already"
Answer: Negative | Shipping Delay Inquiry

Message: "Love the new interface! Very smooth and intuitive"
Answer: Positive | Positive Feedback

Message: "Can you add a dark mode feature to the app?"
Answer: Neutral | Feature Request

Message: "Your product is so good, I am lucky to get it!"
Answer: Positive | Positive Feedback

Message: "The item arrived damaged and has scratches all over"
Answer: Negative | Product Defect

Now classify this message:
Message: "{message}"
Answer:"""

    # Tokenize input
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        max_length=512,
        truncation=True
    ).to(device)

    # Generate output
    outputs = model.generate(
        **inputs,
        max_length=50,
        min_length=5,
        temperature=0.1,
        do_sample=False,
        num_beams=1
    )

    # Decode output
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return result

print("✅ Classification function ready with CORRECT format!")

✅ Classification function ready with CORRECT format!


# Quick Test - single message classification

In [33]:
# Test with a single message
test_message = "My package arrived broken and I want a refund immediately"

print("=" * 70)
print("TEST MESSAGE:")
print(test_message)
print("=" * 70)
print("MODEL OUTPUT:")
print(classify_message(test_message))
print("=" * 70)

TEST MESSAGE:
My package arrived broken and I want a refund immediately
MODEL OUTPUT:
Negative | Product Defect


# Batch Testing - multiple messages

In [34]:
# Test messages covering all categories - CORRECT FORMAT EXPECTED
test_messages = [
    # Product Defect Examples (5 examples) - All NEGATIVE
    "The item arrived damaged and has scratches all over",
    "My laptop screen is flickering and has dead pixels after 3 days",
    "The shoes I received have uneven soles and hurt my feet",
    "Product stopped working after one week, seems like a manufacturing defect",
    "The jacket zipper is broken and one button is missing",

    # Shipping Delay Inquiry Examples (5 examples) - Mostly NEGATIVE
    "It's been 7 days and my order still hasn't arrived",
    "Where is my tracking number? I ordered 3 weeks ago",
    "My package shows delivered but I never received it",
    "When will my order arrive? Just checking the estimated date",
    "Can you update me on my shipment status? It's been 10 days",

    # Mixed Other Categories (5 examples) - POSITIVE, NEGATIVE, NEUTRAL
    "Your product is so good, I am lucky to get it!",
    "I love your new update, the interface is amazing!",
    "The website crashes every time I try to checkout",
    "Can you add a dark mode feature to the mobile app?",
    "How can I change my account password?"
]

print("BATCH TESTING - 15 MESSAGES")
print("=" * 70)
print("Expected Format: Sentiment | Category")
print("=" * 70)

for i, msg in enumerate(test_messages, 1):
    print(f"\n{i}. MESSAGE: {msg}")
    result = classify_message(msg)
    print(f"   OUTPUT: {result}")
    print("-" * 70)



BATCH TESTING - 15 MESSAGES
Expected Format: Sentiment | Category

1. MESSAGE: The item arrived damaged and has scratches all over
   OUTPUT: Negative | Product Defect
----------------------------------------------------------------------

2. MESSAGE: My laptop screen is flickering and has dead pixels after 3 days
   OUTPUT: Negative | Product Defect
----------------------------------------------------------------------

3. MESSAGE: The shoes I received have uneven soles and hurt my feet
   OUTPUT: Negative | Product Defect
----------------------------------------------------------------------

4. MESSAGE: Product stopped working after one week, seems like a manufacturing defect
   OUTPUT: Negative | Product Defect
----------------------------------------------------------------------

5. MESSAGE: The jacket zipper is broken and one button is missing
   OUTPUT: Negative | Product Defect
----------------------------------------------------------------------

6. MESSAGE: It's been 7 days